Importing Dependencies

In [ ]:
import os 
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
dataframe = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv' ))

In [ ]:
dataframe

In [ ]:
dataframe.columns[2:]

Preprocessing

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
x = dataframe['comment_text']
y= dataframe[dataframe.columns[2:]].values

In [ ]:
x

In [ ]:
y

In [ ]:
MAX_FEATURES = 200000 #No of words in the vocab

In [ ]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')

In [ ]:
vectorizer.adapt(x.values)

In [ ]:
vectorized_text = vectorizer(x.values)

In [ ]:
vectorized_text

In [ ]:
#MCSHBAP map, cache, shuffle, batch, prefetch

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [ ]:
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.take(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

Checking the size of partitions

In [ ]:
print(f'Length of train is:{len(train)}, Length of validation is:{len(val)}, Length of test is:{len(test)}')

In [ ]:
train.as_numpy_iterator().next()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [ ]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [ ]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [ ]:
model.summary()

In [ ]:
history = model.fit(train, epochs=10, validation_data=val)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'], color='blue', label='loss')
plt.plot(history.history['val_loss'], color='red', label='val_loss')
plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
input_text = vectorizer("You don't suck at all")

In [ ]:
res = model.predict(np.expand_dims(input_text, 0))

In [ ]:
labels = dataframe.columns
labels[2:]

In [ ]:
batch = test.as_numpy_iterator().next()

In [ ]:
batch_x, batch_y = batch

In [ ]:
batch_y

In [ ]:
(model.predict(batch_x) > 0.5).astype(int)

In [ ]:
(res > 0.5).astype(int)

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
precision = Precision()
recall = Recall()
accuracy = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    x_true, y_true = batch
    
    yhat = model.predict(x_true)

    y_true = y_true.flatten()
    yhat = yhat.flatten()

    precision.update_state(y_true, yhat)
    recall.update_state(y_true, yhat)
    accuracy.update_state(y_true, yhat)

In [ ]:
print(f'Precision{precision.result().numpy()}, Recall:{recall.result().numpy()}, Accuracy:{accuracy.result().numpy()},')

In [ ]:
import gradio as gr

In [ ]:
model.save('toxicity.h5')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
toxicity = load_model('toxicity.h5')

In [ ]:
toxicity.summary()

In [ ]:
input_str = vectorizer('You freaking suck at this game')

In [ ]:
res = model.predict(np.expand_dims(input_str, 0))

In [ ]:
(res > 0.5).astype(int)

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(dataframe.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [ ]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [ ]:
interface.launch(share=True)

Saving vectorizer model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
textvect = tf.keras.models.Sequential()
textvect.add(tf.keras.Input(shape=(1,), dtype=tf.string))
textvect.add(vectorizer)

In [ ]:
textvect.save('vectorizer.tf')

In [ ]:
loaded_vect_model = load_model('vectorizer.tf')
vect_model = loaded_vect_model.layers[0]

In [ ]:
print(vect_model(x.values))

In [ ]:
print(vectorizer(x.values))